<a href="https://colab.research.google.com/github/Ikuesan/Projects/blob/main/Facial_recognition_with_a_pre_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
import zipfile

In [34]:
zip_path = "/content/archive (8).zip"
extract_path = "/content/emotions_data"

In [35]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [36]:
print(os.listdir(extract_path))

['test', 'train']


In [37]:
train_dir = os.path.join(extract_path, 'train')
test_dir = os.path.join(extract_path, 'test')

In [38]:
print(os.listdir(train_dir))

['fear', 'disgust', 'angry', 'neutral', 'sad', 'happy', 'surprise']


In [39]:
# Set image size and batch size
img_height = 224 #224 by 224 because of the pre-trained model required format
img_width = 224
batch_size = 32

In [40]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'  # for multi-class classification
)

Found 28709 files belonging to 7 classes.


In [41]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)

Found 7178 files belonging to 7 classes.


In [42]:
#prefetch the data for faster training time
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [43]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.3),
    layers.RandomTranslation(0.1, 0.1)
])

In [44]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False  # freeze base layers

In [45]:
# Add custom top layers
model = tf.keras.Sequential([
    data_augmentation,                        # data augmentation happens here
    layers.Lambda(tf.keras.applications.mobilenet_v2.preprocess_input),  #preprocessing layer for MobileNetV2
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')  # 7 emotion classes
])

In [46]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [47]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20
)

Epoch 1/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 66s 68ms/step - accuracy: 0.2881 - loss: 1.7885 - val_accuracy: 0.3869 - val_loss: 1.5701
Epoch 2/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 90s 77ms/step - accuracy: 0.3527 - loss: 1.6276 - val_accuracy: 0.4184 - val_loss: 1.4953
Epoch 3/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 73s 67ms/step - accuracy: 0.3772 - loss: 1.6018 - val_accuracy: 0.4479 - val_loss: 1.4591
Epoch 4/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 69s 77ms/step - accuracy: 0.3816 - loss: 1.5769 - val_accuracy: 0.4413 - val_loss: 1.4535
Epoch 5/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 83s 78ms/step - accuracy: 0.3811 - loss: 1.5724 - val_accuracy: 0.4614 - val_loss: 1.4290
Epoch 6/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 70s 78ms/step - accuracy: 0.3937 - loss: 1.5544 - val_accuracy: 0.4570 - val_loss: 1.4240
Epoch 7/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 82s 78ms/step - accuracy: 0.3923 - loss: 1.5509 - val_accuracy: 0.4556 - val_loss: 1.4378
Epoch 8/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 60s 67ms/step - accuracy: 0.3931 - loss: 1.5417 - 

In [30]:
base_model.trainable = True  # now allow training

# Optionally freeze *most* layers, train just the last few
fine_tune_at = 100  # unfreeze from layer 100 onward
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [31]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),  # slow fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [32]:
fine_tune_history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=15
)

Epoch 1/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 125s 120ms/step - accuracy: 0.4012 - loss: 1.5657 - val_accuracy: 0.1967 - val_loss: 3.9690
Epoch 2/15
368/898 ━━━━━━━━━━━━━━━━━━━━ 50s 96ms/step - accuracy: 0.5341 - loss: 1.2792

KeyboardInterrupt: 